In [1]:
import numpy as np
import pandas as pd
import copy

from hongxiongmao import utilities
from hongxiongmao import download
from hongxiongmao import overplot
dl = download.quandl_hxm()

import plotly.offline as py
import plotly_express as pyex
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

In [2]:
cli = dl.from_tickerdict('oecd_cli', start_date='-20y') - 100
gdp = dl.from_tickerdict('oecd_normalised_gdp', start_date='-20y') - 100

In [3]:
df = pd.concat([cli.loc[:,'US'], gdp.loc[:,'US']], axis=1, join='inner')
df.columns = ['cli', 'ref']
df.tail()

,cli,ref
Date,,
2018-12-31,-0.48648,0.2844
2019-01-31,-0.67959,0.2990
2019-02-28,-0.82941,0.3137
2019-03-31,-0.93383,0.3271
2019-04-30,-1.00853,NaN


def swirlygram_animated(df, n=3, lead=1, trail=12, animation=True):
    """
    Animated Swirlygram Function
    
    Swirlygrams are designed to show the amplitude (above or below zero) vs. the periodic change. The swirl coming from a
    timeseries tail. For the animated version we show the swirlygram through time.

    This function looks at a specific case - originally to show the OECD CLI (lagged) vs. OECD Normalised GDP. 
    Function takes a timeseries dataframe with col 0 as a leading indicator & col 1 as the reference variable.
    We then create two reference dataframes which are timeseries of x & y for the CLI and the Ref series, incorporating 
    the lag variable.
    
    NB/ This is a stepping stone to create the more general case... ideally with buttons for multiple plots at the same time
    
    """
    
    ### Default Setup
    cmap=overplot.COLOURMAP.copy()
    layout=copy.deepcopy(overplot.DEFAULT_LAYOUT)
    layout=overplot._update_layout(layout, xaxis={'autorange':False}, yaxis={'autorange':False})
    data=[]
    frames=[]
    
        
    ### Data Manipulation
    
    # Subset Leading Indicator & Reference Index
    # Each has columns x (change) & y (absolute)
    cli = pd.DataFrame(data={'x':df.iloc[:,0]-df.iloc[:,0].shift(n),'y':df.iloc[:,0]}).shift(lead)
    ref = pd.DataFrame(data={'x':df.iloc[:,1]-df.iloc[:,1].shift(n),'y':df.iloc[:,1]})
    
    # Remove stuff
    
    ### Basic Traces
                
    # MARKER for CLI
    data.append(dict(type='scatter', name='lead', mode='markers', showlegend=False, 
                     x=[cli['x'].iloc[-1]],
                     y=[cli['y'].iloc[-1]],
                     marker=dict(symbol='diamond', color=cmap[0], size=10, line={'color':'black', 'width':1}),
                     ))
    
    # MARKER for Ref
    data.append(dict(type='scatter', name='lead', mode='markers', showlegend=False, 
                     x=[ref['x'].iloc[-1]],
                     y=[ref['y'].iloc[-1]],
                     marker=dict(symbol='diamond', color=cmap[1], size=10, line={'color':'black', 'width':1}),
                     ))
    
    # LINE CLI
    data.append(dict(type='scatter', name='lead_line', mode='lines+markers', showlegend=True,
                     x=cli['x'].iloc[-trail:], y=cli['y'].iloc[-trail:],
                     line=dict(color=cmap[0], width=1),
                     ))
    
    # LINE REF
    data.append(dict(type='scatter', name='ref_line', mode='lines+markers', showlegend=True,
                     x=ref['x'].iloc[-trail:], y=ref['y'].iloc[-trail:],
                     line=dict(color=cmap[1], width=1),
                     ))
    
    sliders = {'yanchor': 'top', 'xanchor': 'left', 
                           'currentvalue': {'font':{'size': 12}, 'prefix':'Date : ', 'xanchor': 'left'},
                           'transition': {'duration': 500, 'easing': 'linear'},
                           'pad': {'b': 0, 't': 25}, 'len': 1, 'x': 0, 'y': 0,
                           'steps':[]}
    
    
    ### Additional Layout Changes
    
    # Quadrants
    layout = overplot._quadrants(layout)
    
    # Symmetrical Plot around zero
    absmax = lambda c, x=cli, y=ref: pd.concat([x.iloc[:,c], y.iloc[:,c]]).abs().max()*1.05 
    layout['xaxis']['range'] = [-absmax(0), absmax(0)]
    layout['yaxis']['range'] = [-absmax(1), absmax(1)]
    
    ### Build Figure
    fig = dict(data=data, layout=layout)
    
    
    ### Animations
    if animation:
        
        # Play/Pause Buttons
        buttons = [{'label': 'Play', 'method': 'animate',
                    'args':[None, {'frame': {'duration': 10, 'redraw': False},
                                   'fromcurrent': True,
                                   'transition': {'duration': 10, 'easing': 'quadratic-in-out'}}],},
                   {'label': 'Pause', 'method': 'animate',
                    'args': [[None], {'frame': {'duration': 10, 'redraw': False},
                                      'mode': 'immediate',
                                      'transition': {'duration': 10}}],}]
        
        fig['layout']['updatemenus'] = [{'buttons': buttons,
                                         'direction': 'left',
                                         'pad': {'r': 0, 't': 0},
                                         'showactive': False,
                                         'type': 'buttons',
                                         'x': 0, 'xanchor': 'left',
                                         'y': 0, 'yanchor': 'bottom'
                                     }]
        
        # Iterate through cli adding data sets to frames for each step in animation
        for i, v in enumerate(cli.index.values[(trail+lead+n):-1]):
            
            label = pd.to_datetime(str(v)).strftime('%m/%y')
            #label = '{%m/%y}'.format()    # string label - used to link slider and frame (data things)

            # Append "frame" dictionary to frames list
            frames.append({'name':i, 'layout':{},
                           'data':[dict(type='scatter', x=[cli['x'].iloc[i]], y=[cli['y'].iloc[i]]), # Marker CLI
                                   dict(type='scatter', x=[ref['x'].iloc[i]], y=[ref['y'].iloc[i]]), # Marker Ref
                                   dict(type='scatter', x=cli['x'].iloc[i-trail+1:i],
                                                        y=cli['y'].iloc[i-trail+1:i]), # Line CLI
                                   dict(type='scatter', x=ref['x'].iloc[i-trail+1:i],
                                                        y=ref['y'].iloc[i-trail+1:i]),
                                  ]})

            # Append a "step" dictionary to steps list in sliders dict
            sliders['steps'].append({'label':label, 'method': 'animate', 
                                     'args':[[i], {'frame': {'duration': 250, 'easing':'linear', 'redraw':False},
                                                   'transition':{'duration': 100, 'easing': 'linear'}}],
                                    })

        fig['frames'] = frames
        fig['layout']['sliders'] = [sliders]        # Append completed sliders dictionary to layout
                
    return fig

fig = swirlygram(df, title='OECD Normalised CLI vs. GDP through time', animation=True)
py.iplot(fig, auto_play=False)

In [4]:
def swirlygram_generalised(df, df2=None, n=3, lead=6, trail=18, animation=False, quadrants=True, **kwargs):
    """
    Generalised Swirlygram with Animations & Multiple series
    
    Swirlygrams show the absolute level (amplitude) of, for example, a leading
    indicator vs. the change in the indicator over n periods, as well as a tail
    for the most recent past observations.
    
    My original specific case was to show the OECD CLI vs. Normalised GDP (lagged).
    Thus if the CLI is infact leading GDP by 4-8 months the animation should show
    both comets following each other around the plot, circling the origin.
    
    This function is a generalised swirlygram, meaning:
        * Allows multiple series CLI on one chart, using buttons to change
        * Allows an animation of the swirlygram through time
        * Allows for a "reference series" to be animated at the same time
    
    INPUTS:
        df - pd.DataFrame where each column is a seperate series of the CLI and
              the index is a timeseries index
        df2 - (default is None) is a dataframe of a reference series to be plotted
              alongside the CLI in the animation. IMPORTANT NOTE, this MUST have
              the same column headers in the same order as df or bad things happen.
        n - periodic change (default == 3). Currently this is the absolute shift
        lead - shift applied to df2 i.e. lead=6 will move June '18 to December '18
        trail - length of history to show in comet tail (default==18)
        quadrants - True (default) | False and adds coloured quadrants around origin
        animation - False(default) | True depends if we are making an animation
        **kwargs - mostly for updating layout titles etc... 
    
    NOTES:
        * Absent df2 CLI's will follow normal colourmap
        * Inc df2 all CLIs will be one colour & reference series another
        * chart is forced symmetric around the origin with minimum size of 1
        * inbuilt auto-scaling of x & y. Be careful not to build chart using series
          of very different amplitudes (unless it's deliberate) because the chart will
          size for the largest one and you could lose resolution on the tighter chart
    
    """
    
    ### Default Setup
    cmap=overplot.COLOURMAP.copy()
    layout=copy.deepcopy(overplot.DEFAULT_LAYOUT)
    layout=overplot._update_layout(layout, **kwargs)
    data, frames=[], []
    sliders = {'yanchor':'top', 'xanchor':'left', 
               'currentvalue': {'font':{'size':12}, 'prefix':'Date: ', 'xanchor':'left'},
               'transition': {'duration': 500, 'easing': 'linear'},
                           'pad': {'b': 0, 't': 25}, 'len': 1, 'x': 0, 'y': 0,
                           'steps':[]}
    
    ### Manipulate Data
    x = (df - df.shift(n))
    y = df
    
    # Handle Ref Dataframe
    if isinstance(df2, pd.DataFrame):
        
        # Where 2nd DataFrame passed, ensure indices match main df
        # Also shift 2nd dataframe by desired lead & forward fill missing data
        df2 = pd.concat([pd.DataFrame(index=df.index), df2.shift(lead)], axis=1, sort=False).ffill()
        x1, y1 = (df2 - df2.shift(n)), df2
        ref = True    # quick flag for later
    
    ### Append Data
    for i, c in enumerate(df.columns):
        
        # NB/ Colours follow colourmap if NOT Ref, just use cmap[0] and cmap[1] if Ref
                
        # MARKER for most recent observation
        data.append(dict(type='scatter', mode='markers', showlegend=False, hoverinfo='skip',
                    x=[x[c].iloc[-1]], y=[y[c].iloc[-1]],
                    marker=dict(symbol='diamond', color=cmap[0 if ref else i], size=10,
                               line={'color':'black', 'width':1}),))
        
        # LINE
        data.append(dict(type='scatter', name=c, mode='lines+markers', showlegend=True,
                    x=x[c].iloc[-trail:],
                    y=y[c].iloc[-trail:],
                    line=dict(color=cmap[0 if ref else i], width=1),))
        
        # Reference Dataframe if provided
        if ref:
            
            # MARKER for most recent observation
            data.append(dict(type='scatter', mode='markers', showlegend=False, hoverinfo='skip',
                        x=[x1[c].iloc[-1]], y=[y1[c].iloc[-1]],
                        marker=dict(symbol='diamond', color=cmap[1], size=10,
                        line={'color':'black', 'width':1}),))
            
            # LINE
            data.append(dict(type='scatter', name=c, mode='lines+markers', showlegend=True,
                        x=x1[c].iloc[-trail:],
                        y=y1[c].iloc[-trail:],
                        line=dict(color=cmap[1], width=1),))
            
        ## Buttons
        # Only required if > 1 column
        if len(df.columns) > 1:
            if i == 0: l = len(data)                   # find no traces on 1st pass
            visible = [False] * l * len(df.columns)    # List of False = Total No of Traces
            visible[(i*l):(i*l)+l] = [True] * l        # Make traces of ith pass visible

            button= dict(label=c, method='update', args=[{'visible': visible},])
            layout['updatemenus'][0]['buttons'].append(button)     # append the button 
    
    ### Additional Layout Changes
    
    # Additional Button to make all visible
    if len(df.columns) > 1:
        visible=[True] * l * len(df.columns) 
        button= dict(label='All', method='update', args=[{'visible': visible},])
        layout['updatemenus'][0]['buttons'].append(button)     # append the button 
    
    # Symmetrical Plot around zero
    absmax = lambda df, x=1: np.max([df.abs().max().max(), x]) * 1.05    
    layout['xaxis']['range'] = [-absmax(x), absmax(x)]
    layout['yaxis']['range'] = [-absmax(y), absmax(y)]
    
    # Rectangles of colour for each quadrant
    if quadrants: layout = overplot._quadrants(layout, x=0, y=0)
            
    ### Create Fig
    fig = dict(data=data, layout=layout)
    
    ### Animations
    if animation:
            
        # Need Play/Pause Buttons
        # Using modified version of plotly example
        playpause=[{'label': 'Play', 'method': 'animate',
                    'args':[None, {'frame':{'duration':0,'redraw': False}, 'fromcurrent': True,
                                   'transition': {'duration':0,'easing':'quadratic-in-out'}}],},
                   {'label': 'Pause', 'method': 'animate',
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode':'immediate',
                                      'transition': {'duration': 0}}],}]
        
        # Append to layout - remembering we may already have a set of buttons in updatemenus
        fig['layout']['updatemenus'].append({'buttons': playpause, 'type': 'buttons', 'showactive': False,
                                             'x':0, 'xanchor':'left', 'y':0, 'yanchor':'bottom',
                                             'direction': 'left', 'pad': {'r': 0, 't': 0},})
        
        ## Build Animation Slider
        
        # Iterate through cli adding data sets to frames for each step in animation
        for i, v in enumerate(x.index.values[trail:-1]):
            
            # Complicated bit is building frame traces for each step
            frame_traces = []
            
            for j, c in enumerate(df.columns):
                # Main markers & lines
                frame_traces.append(dict(type='scatter', x=[x.iloc[i,j]], y=[y.iloc[i,j]]))    # marker
                frame_traces.append(dict(type='scatter', x=x.iloc[i-trail:i, j],               # line
                                                         y=y.iloc[i-trail:i, j]))
                
                if ref:
                    # Only add these if reference series required
                    frame_traces.append(dict(type='scatter', x=[x1.iloc[i,j]], y=[y1.iloc[i,j]]))
                    frame_traces.append(dict(type='scatter', x=x1.iloc[i-trail:i, j], y=y1.iloc[i-trail:i, j]))
            
            
            # Append "frame" dictionary to frames list
            frames.append({'name':i, 'layout':{}, 'data':frame_traces})

            # Append a "step" dictionary to steps list in sliders dict
            label = pd.to_datetime(str(v)).strftime('%m/%y')    # String Date Label
            sliders['steps'].append({'label':label, 'method': 'animate', 
                                     'args':[[i], {'frame': {'duration': 500, 'easing':'linear', 'redraw':False},
                                                   'transition':{'duration': 25, 'easing': 'linear'}}],})
            
        fig['frames'] = frames
        fig['layout']['sliders'] = [sliders]        # Append completed sliders dictionary to layout
                
    return fig

fig = swirlygram_generalised(cli.iloc[:,0:3], gdp.iloc[:,0:3], animation=True)
py.iplot(fig)